In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append("../../framework")
sys.path.append("../../concrete")
sys.path.append("../../snippet")


In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import  ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

# 10. Misc

## 12. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## 14. a function to load gain and bias from the trains over a training:

In [ ]:
def getGainAndBiasOverTrain(buildOrderIdGiven):

    concStore = ConcStore()
    agentFactory = ConcAgentFactory()

    y = np.random.randn(1, ConcEnvironment.nPv).astype(np.float32)
    anObservationSequence = ObservationSequence()
    anObservationSequence.add(ConcObservation(y))

    cur.execute('''
    Select
        count(*)
        From TrainLog
        Where buildOrderId = ?
        ''', (buildOrderIdGiven,))
    assert cur.fetchone()[0] > 0, "NO DATA WITH THE GIVEN BUILD ID"

    cur.execute('''
    Select 
        trainId
        , timeSimulation
        , timestamp
        From TrainLog
        Where buildOrderId = ?
        Order By timeSimulation
        ''', (buildOrderIdGiven,))


    TimeSimulation = []
    Gain = []
    Bias = []

    for (trainIdStr, timeSimulation, timestamp) in cur.fetchall():

        # storeField contains: agentMement, buildOrder
        storeField = concStore.load(TrainId(trainIdStr))

        # restore agent
        agent = agentFactory.create(storeField.getBuildOrder())
        agent.loadFromMemento(storeField.getAgentMemento())
        agent(anObservationSequence)

        TimeSimulation.append(timeSimulation)
        
        gain = agent.gainP.weights[0].numpy()
        if len(agent.gainP.weights) > 1:
            bias = agent.gainP.weights[1].numpy()
        else:
            bias = np.nan
            
        Gain.append(gain)
        Bias.append(bias)

    TimeSimulation = np.array(TimeSimulation) # (*)
    Gain = np.concatenate(Gain, axis=0) # (*, nMv)
    Bias = np.stack(Bias, axis=0) # (*, nMv)
    
    return TimeSimulation, Gain, Bias

# 20. Observe the trace of gain and bias over a traing

## 22. Choose a build order

In [ ]:
cur.execute('''
Select
    Distinct buildOrderId
    From TrainLog
     Where buildOrderId like "test004%"
    ''')

buildOrderIds = [elm for elm, in cur.fetchall()]

## 24. Load the parameters => SKIPPED

In [ ]:
# TimeSimulation, Gain, Bias = getGainAndBiasOverTrain(buildOrderIdGiven)

## 26. Plot the trace of Gain and Bias along with the simulation time => SKIPPED

In [ ]:
# fig = plt.figure()
# fig.clf()
# #
# ax = fig.add_subplot(2,1,1)
# ax.plot(TimeSimulation, Gain, 'o-')
# ax.set_title('Gain')
# #
# ax = fig.add_subplot(2,1,2)
# ax.plot(TimeSimulation, Bias, 'o-')
# ax.set_title('Bias')
# #
# plt.tight_layout()

## 28. plot all the trend of parameters

In [ ]:
fig = plt.figure()
fig.clf()
#

ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)

ax1.set_ylabel('Gain')
ax2.set_ylabel('Bias')

for buildOrderIdGiven in buildOrderIds:
    TimeSimulation, Gain, Bias = getGainAndBiasOverTrain(buildOrderIdGiven)

    ax1.plot(TimeSimulation, Gain, 'k-', label = buildOrderIdGiven)
    ax2.plot(TimeSimulation, Bias, 'k-', label = buildOrderIdGiven)

#
ax1.legend()
ax2.legend()
plt.tight_layout()